Download all the required packages

In [10]:
! pip install bs4
!pip install lxml
!pip install html5lib

In [2]:
from bs4 import BeautifulSoup 
import requests
import lxml
import pandas as pd
import numpy as np
import html5lib
import io

Scrape Wikipedia for data <br>
Import location data from cvs online

In [3]:
html=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
url2 = 'http://cocl.us/Geospatial_data'

Format Wikipedia postcode data

In [5]:
soup = BeautifulSoup(html, 'html5lib')
table = soup.find('table_class',{'class':'navbox'})

In [6]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
      
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
    pass

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})


Format Location data

In [7]:
postcode_table = requests.get(url2).content
postcode_lat_long = pd.read_csv(io.StringIO(postcode_table .decode('utf-8')))

Join Postcode data and location data

In [8]:
Toronto = pd.merge(df, postcode_lat_long, left_on = "PostalCode", right_on="Postal Code" )
Toronto= Toronto.drop(["Postal Code"], axis= 1)

Display the shape of the data

In [11]:
Toronto.shape

(103, 5)